In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from typing import Tuple, Dict

import requests_cache
import openmeteo_requests
from retry_requests import retry

from rich.console import Console
from rich.table import Table
from rich.panel import Panel

console = Console()

# Weather cache directory
WEATHER_CACHE_DIR = Path("data/weather_cache")
WEATHER_CACHE_DIR.mkdir(parents=True, exist_ok=True)

START_DATE = "2020-01-01"
END_DATE = datetime.now().strftime("%Y-%m-%d")


In [ ]:
def get_unique_locations(event_df: pd.DataFrame) -> list[tuple]:
    """
    Returns list of unique (lat, lon) pairs.
    Filters out missing or invalid coordinates.
    """
    coords = (
        event_df[["latitude", "longitude"]]
        .dropna()
        .drop_duplicates()
        .itertuples(index=False, name=None)
    )
    return list(coords)


In [ ]:
def fetch_weather_for_location(lat: float, lon: float) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Fetch historical hourly + daily weather for a single location.
    """

    console.print(
        Panel(
            f"[bold cyan]Fetching Weather for Location[/bold cyan]\nLat={lat:.3f}, Lon={lon:.3f}",
            border_style="cyan",
        )
    )

    cache_session = requests_cache.CachedSession(".weather_cache", expire_after=-1)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.3)
    client = openmeteo_requests.Client(session=retry_session)

    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": START_DATE,
        "end_date": END_DATE,
        "hourly": [
            "temperature_2m",
            "precipitation",
            "rain",
            "apparent_temperature",
            "weather_code",
            "is_day",
        ],
        "daily": [
            "sunrise",
            "daylight_duration",
            "sunshine_duration",
            "precipitation_hours",
            "rain_sum",
            "temperature_2m_mean",
            "weather_code",
        ],
        "timezone": "America/New_York",
        "temperature_unit": "fahrenheit",
    }

    responses = client.weather_api(url, params=params)
    response = responses[0]

    # -------- HOURLY DATA --------
    hourly = response.Hourly()
    hourly_df = pd.DataFrame({
        "datetime": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left",
        ),
        "temp_f": hourly.Variables(0).ValuesAsNumpy(),
        "precip_in": hourly.Variables(1).ValuesAsNumpy(),
        "rain_in": hourly.Variables(2).ValuesAsNumpy(),
        "apparent_temp_f": hourly.Variables(3).ValuesAsNumpy(),
        "weather_code_hourly": hourly.Variables(4).ValuesAsNumpy(),
        "is_daylight": hourly.Variables(5).ValuesAsNumpy().astype(int),
    })

    hourly_df["datetime"] = hourly_df["datetime"].dt.tz_convert("America/New_York").dt.tz_localize(None)

    # -------- DAILY DATA --------
    daily = response.Daily()
    daily_df = pd.DataFrame({
        "date": pd.date_range(
            start=pd.to_datetime(daily.Time(), unit="s", utc=True),
            end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=daily.Interval()),
            inclusive="left",
        ),
        "sunrise": daily.Variables(0).ValuesInt64AsNumpy(),
        "daylight_duration_sec": daily.Variables(1).ValuesAsNumpy(),
        "sunshine_duration_sec": daily.Variables(2).ValuesAsNumpy(),
        "precip_hours": daily.Variables(3).ValuesAsNumpy(),
        "rain_sum_in": daily.Variables(4).ValuesAsNumpy(),
        "temp_mean_f": daily.Variables(5).ValuesAsNumpy(),
        "weather_code_daily": daily.Variables(6).ValuesAsNumpy(),
    })

    daily_df["date"] = daily_df["date"].dt.tz_convert("America/New_York").dt.tz_localize(None).dt.date

    return hourly_df, daily_df


In [ ]:
def load_or_fetch_weather(lat: float, lon: float) -> Tuple[pd.DataFrame, pd.DataFrame]:

    safe_lat = f"{lat:.3f}".replace(".", "_")
    safe_lon = f"{lon:.3f}".replace(".", "_")
    base = WEATHER_CACHE_DIR / f"weather_{safe_lat}_{safe_lon}"

    hourly_file = base.with_suffix(".hourly.parquet")
    daily_file = base.with_suffix(".daily.parquet")

    if hourly_file.exists() and daily_file.exists():
        console.print(f"[green]Using cached weather for ({lat}, {lon})[/green]")
        return (
            pd.read_parquet(hourly_file),
            pd.read_parquet(daily_file)
        )

    hourly_df, daily_df = fetch_weather_for_location(lat, lon)

    hourly_df.to_parquet(hourly_file, index=False)
    daily_df.to_parquet(daily_file, index=False)

    console.print(f"[green]Cached weather for ({lat}, {lon})[/green]")

    return hourly_df, daily_df


In [ ]:
def fetch_weather_for_all_event_locations(event_df: pd.DataFrame) -> Dict[tuple, dict]:
    """
    Fetches or loads cached weather for all unique venue coordinates.
    """

    locations = get_unique_locations(event_df)

    console.print(
        Panel(
            f"[cyan]Fetching Weather for {len(locations)} Unique Venue Locations[/cyan]",
            border_style="cyan"
        )
    )

    weather_map = {}

    for lat, lon in locations:
        hourly, daily = load_or_fetch_weather(lat, lon)
        weather_map[(lat, lon)] = {"hourly": hourly, "daily": daily}

    return weather_map


In [ ]:
def merge_event_weather(event_df: pd.DataFrame, weather_map: Dict[tuple, dict]) -> pd.DataFrame:

    df = event_df.copy()
    df["event_date"] = pd.to_datetime(df["event_date"], errors="coerce")

    df_weather = []

    # Group events by location to speed up merging
    for (lat, lon), group in df.groupby(["latitude", "longitude"]):

        if (lat, lon) not in weather_map:
            console.print(f"[yellow]No weather found for ({lat}, {lon})[/yellow]")
            df_weather.append(group)
            continue

        hourly_w = weather_map[(lat, lon)]["hourly"].copy()
        daily_w = weather_map[(lat, lon)]["daily"].copy()

        hourly_w["weather_hour"] = pd.to_datetime(hourly_w["datetime"])
        daily_w["weather_date"] = pd.to_datetime(daily_w["date"]).dt.date

        temp = group.copy()

        # HOURLY MERGE
        temp["event_hour"] = temp["event_date"].dt.floor("h")
        temp = temp.merge(hourly_w.drop(columns=["datetime"]), left_on="event_hour", right_on="weather_hour", how="left")

        # DAILY MERGE
        temp["event_day"] = temp["event_date"].dt.date
        temp = temp.merge(daily_w.drop(columns=["date"]), left_on="event_day", right_on="weather_date", how="left")

        df_weather.append(temp)

    merged = pd.concat(df_weather, ignore_index=True)

    # Clean helper columns
    merged.drop(columns=["event_hour", "weather_hour", "event_day", "weather_date"], errors="ignore", inplace=True)

    console.print("[green]Weather successfully merged into event data![/green]")

    return merged


In [ ]:
def add_weather_flags(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    if "temp_f" not in df.columns:
        console.print("[yellow]No temp_f column found — skipping flags.[/yellow]")
        return df

    p85 = df["temp_f"].dropna().quantile(0.85)
    p15 = df["temp_f"].dropna().quantile(0.15)

    df["is_hot"] = (df["temp_f"] >= p85).astype(int)
    df["is_cold"] = (df["temp_f"] <= p15).astype(int)
    df["is_raining"] = (df.get("precip_in", 0) > 0.01).astype(int)

    return df


In [ ]:
def build_weather_event_panel(event_json_path: str, out_path: str):
    console.print(Panel("[bold cyan]Building Weather-Enriched Event Panel[/bold cyan]"))

    df_events = pd.read_json(event_json_path)

    # Ensure numeric coords
    df_events["latitude"] = pd.to_numeric(df_events["latitude"], errors="coerce")
    df_events["longitude"] = pd.to_numeric(df_events["longitude"], errors="coerce")

    console.print(f"[blue]Loaded {len(df_events):,} events[/blue]")

    # Fetch + cache weather
    weather_map = fetch_weather_for_all_event_locations(df_events)

    # Merge weather into events
    enriched = merge_event_weather(df_events, weather_map)

    # Add flags
    enriched = add_weather_flags(enriched)

    # Save
    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    enriched.to_parquet(out_path, index=False)

    console.print(f"[green]Saved enriched panel → {out_path}[/green]")

    return enriched


In [ ]:
enriched_df = build_weather_event_panel(
    event_json_path="data/raw/ticketmaster_us_music_latest.json",
    out_path="data/processed/event_weather_panel.parquet",
)

enriched_df.head()
